In [1]:
import os
from dotenv import load_dotenv

import pandas as pd

from h2ogpte import H2OGPTE

In [2]:
df = pd.read_excel('../QandA.xlsx')
print(df.shape)
df.head()

(25, 5)


,No,Question,Answer,ページ,コメント
0,1,ビールの主な原材料を全て教えてください？,麦芽、ホップ、酵母、水、副原料,4,NaN
1,2,ビールの製造工程の3番目の工程は？,煮沸,5,NaN
2,3,Craft Beer Marketでは何種類のビールを提供していますか？,9種類,"6,7",NaN
3,4,Craft Beer Marketが提供するビールの中で、アルコール度数の最も高いと考えられ...,SPECIAL BEER,7,NaN
4,5,代表的なエールスタイルのビールを３つ教えてください？,「アメリカン・ペールエール」「イングリッシュ・ペールエー\nル」「アンバーエール」「レッドエ...,6,NaN


In [3]:
# System Prompts

system_prompt = '''
あなたは質問に対し、与えられたドキュメントから回答を実施する優秀なRAGチャットアシスタントです。
必ず日本語で回答します。
長くても70トークン以内で簡潔に回答を作成してください。回答を得た理由を述べる必要はないです。
'''
pre_prompt_query='以下の情報に注意して覚えておいてください。最後に質問や命令に答えるためには、チャット履歴、指定された画像、または文書テキストのみを使用する必要があります。'
prompt_query='チャット履歴の情報のみに従って、与えられた画像、または上記のコンテキスト内で提供されているドキュメントテキストは、次のように明確に構成された応答（「与えられた情報によると、」から始まる）を示します。'

## Collection: Craft-Beer-Market_default

In [6]:
load_dotenv(dotenv_path='.env')
heogpteKey_CBM_default = os.getenv('CBM_default')
if heogpteKey_CBM_default: print('key is set.')

client = H2OGPTE(
    address='https://h2ogpte.dev.h2o.ai',
    api_key=heogpteKey_CBM_default,
)
client

key is set.


### LLM: gpt-4o-min

In [23]:
%%time

LlmModel = 'gpt-4o-mini'    # この部分のみ置き換える

chat_session_id = client.create_chat_session_on_default_collection()
print(chat_session_id)

res = []

#for i, q in enumerate(df['Question'][:3]):
for i, q in enumerate(df['Question']):
    print('No: ', i+1)
    print('Question > ', q)

    with client.connect(chat_session_id) as session:
        reply = session.query(
            message = q,
            system_prompt=system_prompt,
            pre_prompt_query=pre_prompt_query,
            prompt_query=prompt_query,
            llm=LlmModel,
            rag_config={"rag_type": "rag"},   # (Normal) RAG 
        )
        print('Answer > ', reply.content)
        res.append(reply.content)

print(len(res))

pd.DataFrame({LlmModel:res}).to_csv('../results/res__{}.csv'.format(LlmModel), index=False)

e1447251-171d-4f8e-95c7-14759716241d
No:  1
Question >  ビールの主な原材料を全て教えてください？
Answer >  与えられた情報によると、ビールの主な原材料は以下の通りです。麦芽（主に二条大麦）、ホップ、酵母、水、そして副原料として米、糖類、コーンスターチ、着色料（カラメル）などが使用されます。
No:  2
Question >  ビールの製造工程の3番目の工程は？
Answer >  与えられた情報によると、ビールの製造工程の3番目は「冷却・酵母添加」です。この工程では、煮沸終了後に麦汁を冷却し、発酵に必要な酸素とともに酵母を加えます。
No:  3
Question >  Craft Beer Marketでは何種類のビールを提供していますか？
Answer >  与えられた情報によると、Craft Beer Marketでは以下のビールスタイルを提供しています：ラガー、エール、ホワイトビール、フルーツビール、黒ビール、IPA、リアルエール、スペシャルビールなどがあります。
No:  4
Question >  Craft Beer Marketが提供するビールの中で、アルコール度数の最も高いと考えられるビールは？
Answer >  与えられた情報によると、Craft Beer Marketが提供するビールの中でアルコール度数が最も高いと考えられるのは、長期熟成でハイアルコールの「バーレーワイン」や「ウィートワイン」、その他のストロングエールです。
No:  5
Question >  代表的なエールスタイルのビールを３つ教えてください？
Answer >  与えられた情報によると、代表的なエールスタイルのビールには「アメリカン・ペールエール」「イングリッシュ・ペールエール」「アンバーエール」があります。
No:  6
Question >  テイスティングで、口当たりと外観はどちらを先に確認しますか？
Answer >  与えられた情報によると、テイスティングの5段階では、まず外観を確認し、その後に口当たりを評価します。
No:  7
Question >  テイスティングの際、どのような条件の時、香りが強くなるか？
Answer >  与えられた情報によると、ビールの温度が高くなると香りが強まり

### LLM: Qwen/Qwen2.5-72B-Instruct

In [29]:
%%time

LlmModel = 'Qwen/Qwen2.5-72B-Instruct'    # この部分のみ置き換える

chat_session_id = client.create_chat_session_on_default_collection()
print(chat_session_id)

res = []

#for i, q in enumerate(df['Question'][:3]):
for i, q in enumerate(df['Question']):
    print('No: ', i+1)
    print('Question > ', q)

    with client.connect(chat_session_id) as session:
        reply = session.query(
            message = q,
            system_prompt=system_prompt,
            pre_prompt_query=pre_prompt_query,
            prompt_query=prompt_query,
            llm=LlmModel,
            rag_config={"rag_type": "rag"},   # (Normal) RAG 
        )
        print('Answer > ', reply.content)
        res.append(reply.content)

print(len(res))

mod_name = LlmModel.replace('/', '--')
pd.DataFrame({LlmModel:res}).to_csv('../results/res__{}.csv'.format(mod_name), index=False)

b44869a4-d93f-4fb0-8970-ae185c28168a
No:  1
Question >  ビールの主な原材料を全て教えてください？
Answer >  まず、ユーザーが「ビールの主な原材料を全て教えてください？」と尋ねています。この質問に答えるためには、与えられたドキュメントから関連する情報を抽出する必要があります。

ドキュメントを確認すると、ページ4に「《ビールの主な原材料》」と題された節があります。この節では、麦芽、ホップ、水、副原料が挙げられています。麦芽については、二条大麦が使用され、品質や価格面で輸入麦芽が優れていることが記載されています。ホップについては、セイヨウカラハナソウの花の部分が使用され、芳香や苦味、防腐作用があると説明されています。水については、無色・透明・無味・無臭で、生物的に汚染されていないことが求められているとあります。副原料には、麦、米、糖類、コーンスターチ、塩、着色料（カラメル）が含まれており、これらを使用すると発泡酒になることも記載されています。

これらの情報を整理し、ユーザーに明確に伝える必要があります。また、日本語で簡潔に回答し、70トークン以内に収める必要があります。
</think>

与えられた情報によると、ビールの主な原材料は麦芽、ホップ、水、副原料です。麦芽は二条大麦を使用し、ホップはセイヨウカラハナソウの花の部分を使用します。副原料には麦、米、糖類、コーンスターチ、塩、カラメルが含まれます。
No:  2
Question >  ビールの製造工程の3番目の工程は？
Answer >  まず、ユーザーが「ビールの製造工程の3番目の工程は？」と尋ねています。この質問に答えるためには、ドキュメント中の製造工程に関する部分を確認する必要があります。

ドキュメントを確認すると、ページ5に「《ビールの簡単な製造工程》」が記載されています。この工程は以下の通りです：

1. 糖化
2. 煮沸
3. 冷却・酵母添加
4. 主発酵・後発酵・ドライホッピング
5. ビン詰め・樽詰め

ユーザーが尋ねているのは3番目の工程です。この工程は「冷却・酵母添加」と呼ばれます。この段階では、煮沸後の麦汁を冷却し、酵母を加えることで発酵を開始します。冷却は発酵のための適切な温度を保つために重要で、酵母の添加は発酵工程の起

### LLM: meta-llama/Meta-Llama-3.1-8B-Instruct

In [30]:
%%time

LlmModel = 'meta-llama/Meta-Llama-3.1-8B-Instruct'    # この部分のみ置き換える

chat_session_id = client.create_chat_session_on_default_collection()
print(chat_session_id)

res = []

#for i, q in enumerate(df['Question'][:3]):
for i, q in enumerate(df['Question']):
    print('No: ', i+1)
    print('Question > ', q)

    with client.connect(chat_session_id) as session:
        reply = session.query(
            message = q,
            system_prompt=system_prompt,
            pre_prompt_query=pre_prompt_query,
            prompt_query=prompt_query,
            llm=LlmModel,
            rag_config={"rag_type": "rag"},   # (Normal) RAG 
        )
        print('Answer > ', reply.content)
        res.append(reply.content)

print(len(res))

mod_name = LlmModel.replace('/', '--')
pd.DataFrame({LlmModel:res}).to_csv('../results/res__{}.csv'.format(mod_name), index=False)

6d739d01-e050-4cc6-abb9-63bd8d84b89c
No:  1
Question >  ビールの主な原材料を全て教えてください？
Answer >  与えられた情報によると、ビールの主な原材料は麦芽、ホップ、酵母、糖です。
No:  2
Question >  ビールの製造工程の3番目の工程は？
Answer >  与えられた情報によると、ビールの製造工程の3番目の工程は「冷却・酵母添加」です。
No:  3
Question >  Craft Beer Marketでは何種類のビールを提供していますか？
Answer >  与えられた情報によると、Craft Beer Marketでは以下のビールを提供しています。

1. KANPAI! STANDARD BEER（グラス：中生タンブラー パイント：UK568ml）
2. LAGER BEER / PILSNER （グラス：250ml パイント：US473ml）
3. ALE BEER （グラス：250ml パイント：US473ml）
4. WHITE BEER / WEIZEN （グラス：250ml パイント：US473ml）
5. IPA – INDIA PALE ALE （グラス：250ml パイント：US473ml）
6. FRUIT/HERB BEER （グラス：250ml パイント：US473ml）
7. BLACK BEER （グラス：250ml パイント：US473ml）
8. SPECIAL BEER （Wine Glass：100ml/¥480 , 200ml/¥780 ）
9. REAL ALE （グラス：250ml パイント：US473ml）
No:  4
Question >  Craft Beer Marketが提供するビールの中で、アルコール度数の最も高いと考えられるビールは？
Answer >  与えられた情報によると、Craft Beer Marketが提供するビールの中で、アルコール度数の最も高いと考えられるビールは「SPECIAL BEER」です。SPECIAL BEERは長期熟成でアルコール度数がハイアルコールのビールであり、「バーレーワイン」や「ウィートワイン」または、その他のストロングエールが該当することが記載されています。
No:  5
Que

### LLM: gpt-4o

In [31]:
%%time

LlmModel = 'gpt-4o'    # この部分のみ置き換える

chat_session_id = client.create_chat_session_on_default_collection()
print(chat_session_id)

res = []

#for i, q in enumerate(df['Question'][:3]):
for i, q in enumerate(df['Question']):
    print('No: ', i+1)
    print('Question > ', q)

    with client.connect(chat_session_id) as session:
        reply = session.query(
            message = q,
            system_prompt=system_prompt,
            pre_prompt_query=pre_prompt_query,
            prompt_query=prompt_query,
            llm=LlmModel,
            rag_config={"rag_type": "rag"},   # (Normal) RAG 
        )
        print('Answer > ', reply.content)
        res.append(reply.content)

print(len(res))

mod_name = LlmModel.replace('/', '--')
pd.DataFrame({LlmModel:res}).to_csv('../results/res__{}.csv'.format(mod_name), index=False)

02951fea-4ad8-4a29-a777-c5280098f3fa
No:  1
Question >  ビールの主な原材料を全て教えてください？
Answer >  与えられた情報によると、ビールの主な原材料は以下の通りです：

1. 麦芽（主に二条大麦）
2. ホップ
3. 水
4. 酵母

また、副原料として麦（ライ麦、小麦など）、米、糖類、コーンスターチ、こうりやん、ばれいしょ、着色料（カラメル）などが使用されることがあります。
No:  2
Question >  ビールの製造工程の3番目の工程は？
Answer >  与えられた情報によると、ビールの製造工程の3番目の工程は「煮沸」です。
No:  3
Question >  Craft Beer Marketでは何種類のビールを提供していますか？
Answer >  与えられた情報によると、Craft Beer Marketでは以下の種類のビールを提供しています：

1. KANPAI! STANDARD BEER
2. LAGER BEER / PILSNER
3. ALE BEER
4. WHITE BEER / WEIZEN
5. FRUIT/HERB BEER
6. BLACK BEER
7. IPA – INDIA PALE ALE
8. REAL ALE
9. SPECIAL BEER

合計で9種類のビールを提供しています。
No:  4
Question >  Craft Beer Marketが提供するビールの中で、アルコール度数の最も高いと考えられるビールは？
Answer >  与えられた情報によると、Craft Beer Marketが提供するビールの中でアルコール度数の最も高いと考えられるビールは「SPECIAL BEER」です。
No:  5
Question >  代表的なエールスタイルのビールを３つ教えてください？
Answer >  与えられた情報によると、代表的なエールスタイルのビールは以下の3つです：
1. アメリカン・ペールエール
2. イングリッシュ・ペールエール
3. アンバーエール
No:  6
Question >  テイスティングで、口当たりと外観はどちらを先に確認しますか？
Answer >  与えられた情報によると、テイスティングでは外観を先に

## Collection: Craft-Beer-Market_structured

In [38]:
#!cat .env

In [39]:
load_dotenv(dotenv_path='.env')
heogpteKey_CBM_structured = os.getenv('CBM_structured')
if heogpteKey_CBM_structured: print('key is set.')

client = H2OGPTE(
    address='https://h2ogpte.dev.h2o.ai',
    api_key=heogpteKey_CBM_structured,
)
client

key is set.


In [40]:
save_dir = 'Craft-Beer-Market_structured'

### LLM: gpt-4o-min

In [44]:
%%time

LlmModel = 'gpt-4o-mini'    # この部分のみ置き換える

chat_session_id = client.create_chat_session_on_default_collection()
print(chat_session_id)

res = []

#for i, q in enumerate(df['Question'][:3]):
for i, q in enumerate(df['Question']):
    print('No: ', i+1)
    print('Question > ', q)

    with client.connect(chat_session_id) as session:
        reply = session.query(
            message = q,
            system_prompt=system_prompt,
            pre_prompt_query=pre_prompt_query,
            prompt_query=prompt_query,
            llm=LlmModel,
            rag_config={"rag_type": "rag"},   # (Normal) RAG 
        )
        print('Answer > ', reply.content)
        res.append(reply.content)

print(len(res))

mod_name = LlmModel.replace('/', '--')
pd.DataFrame({LlmModel:res}).to_csv('../results/{}/res__{}.csv'.format(save_dir, mod_name), index=False)

d0cf92f1-cdf1-40c1-9005-dd768042b8a3
No:  1
Question >  ビールの主な原材料を全て教えてください？
Answer >  与えられた情報によると、ビールの主な原材料は以下の通りです：
1. ホップ
2. 麦芽
3. 水
4. 酵母
これらの原材料がビールの色や味わい、泡の形成に影響を与えます。
No:  2
Question >  ビールの製造工程の3番目の工程は？
Answer >  与えられた情報によると、ビールの製造工程の3番目は「煮沸」です。この工程では、麦汁を煮沸し、ホップを加えて苦味をつけます。
No:  3
Question >  Craft Beer Marketでは何種類のビールを提供していますか？
Answer >  与えられた情報によると、Craft Beer Marketでは以下のビアスタイルを提供しています：ラガー、エール、ホワイトビール、フルーツ/ハーブビール、IPA、リアルエール、スペシャルビールなどがあります。
No:  4
Question >  Craft Beer Marketが提供するビールの中で、アルコール度数の最も高いと考えられるビールは？
Answer >  与えられた情報によると、Craft Beer Marketが提供するビールの中で、アルコール度数が最も高いと考えられるのは「SPECIAL BEER」で、長期熟成でハイアルコールのビールです。
No:  5
Question >  代表的なエールスタイルのビールを３つ教えてください？
Answer >  与えられた情報によると、代表的なエールスタイルのビールには「アメリカン・ペールエール」「イングリッシュ・ペールエール」「ブラウンエール」があります。
No:  6
Question >  テイスティングで、口当たりと外観はどちらを先に確認しますか？
Answer >  与えられた情報によると、テイスティングではまず「外観」を確認し、その後「口当たり」を評価します。
No:  7
Question >  テイスティングの際、どのような条件の時、香りが強くなるか？
Answer >  与えられた情報によると、ビールの温度が高くなると香りが高まります。ビールを真上から軽く嗅ぐことで、原材料からアロマを確認することができま

### Qwen/Qwen2.5-72B-Instruct

In [45]:
%%time

LlmModel = 'Qwen/Qwen2.5-72B-Instruct'    # この部分のみ置き換える

chat_session_id = client.create_chat_session_on_default_collection()
print(chat_session_id)

res = []

#for i, q in enumerate(df['Question'][:3]):
for i, q in enumerate(df['Question']):
    print('No: ', i+1)
    print('Question > ', q)

    with client.connect(chat_session_id) as session:
        reply = session.query(
            message = q,
            system_prompt=system_prompt,
            pre_prompt_query=pre_prompt_query,
            prompt_query=prompt_query,
            llm=LlmModel,
            rag_config={"rag_type": "rag"},   # (Normal) RAG 
        )
        print('Answer > ', reply.content)
        res.append(reply.content)

print(len(res))

mod_name = LlmModel.replace('/', '--')
pd.DataFrame({LlmModel:res}).to_csv('../results/{}/res__{}.csv'.format(save_dir, mod_name), index=False)

5ac2fe77-beaa-4ea2-8c0d-41d2c9ea6377
No:  1
Question >  ビールの主な原材料を全て教えてください？
Answer >  まず、ユーザーが「ビールの主な原材料を全て教えてください？」と尋ねています。この質問に答えるためには、ドキュメントから関連する情報を抽出する必要があります。

ドキュメントを見ると、名前が「1_ビールについて.pdf」の文書に原材料に関する情報があります。この文書のページ1とページ2に、ビールの主な原材料としてホップ、麦芽、水、酵母、副原料が説明されています。

ホップについては、つる性植物で和名はセイヨウカラハナソウと説明されています。麦芽は二条大麦を使用し、ビールの色や味に影響を与えるとあります。水は原料の大部分を占め、適質な水が重要だと説明されています。酵母は糖を分解してアルコールと炭酸ガスを生成する微生物で、ラガー酵母とエール酵母の2種類が説明されています。副原料として、麦、米、糖類、コーンスターチ、カラメルなどが挙げられています。

これらの情報を整理し、ユーザーに分かりやすく伝える必要があります。また、回答は日本語で簡潔に70トークン以内に収める必要があります。
</think>

与えられた情報によると、ビールの主な原材料はホップ、麦芽、水、酵母、副原料です。
No:  2
Question >  ビールの製造工程の3番目の工程は？
Answer >  まず、ユーザーが「ビールの製造工程の3番目の工程は？」と尋ねています。この質問に答えるためには、ドキュメント中の「1_ビールについて.pdf」を見つける必要があります。このドキュメントのページ1と2には製造工程の詳細が記載されています。

ドキュメントを確認すると、製造工程は以下の順番で説明されています：
1. 糖化
2. ろ過
3. 煮沸
4. 冷却・酵母添加
5. 主発酵・後発酵・ドライホッピング

ユーザーが尋ねているのは3番目の工程です。この工程は「煮沸」と呼ばれます。煮沸の工程では、麦汁を煮沸し、ホップを加えて苦味をつけることが説明されています。

したがって、3番目の工程は煮沸です。
</think>

与えられた情報によると、ビールの製造工程の3番目の工程は煮沸です。
No:  3
Question >  Craft 

### meta-llama/Meta-Llama-3.1-8B-Instruct

In [46]:
%%time

LlmModel = 'meta-llama/Meta-Llama-3.1-8B-Instruct'    # この部分のみ置き換える

chat_session_id = client.create_chat_session_on_default_collection()
print(chat_session_id)

res = []

#for i, q in enumerate(df['Question'][:3]):
for i, q in enumerate(df['Question']):
    print('No: ', i+1)
    print('Question > ', q)

    with client.connect(chat_session_id) as session:
        reply = session.query(
            message = q,
            system_prompt=system_prompt,
            pre_prompt_query=pre_prompt_query,
            prompt_query=prompt_query,
            llm=LlmModel,
            rag_config={"rag_type": "rag"},   # (Normal) RAG 
        )
        print('Answer > ', reply.content)
        res.append(reply.content)

print(len(res))

mod_name = LlmModel.replace('/', '--')
pd.DataFrame({LlmModel:res}).to_csv('../results/{}/res__{}.csv'.format(save_dir, mod_name), index=False)

edf70c68-5c2e-4333-a310-58ad25024ceb
No:  1
Question >  ビールの主な原材料を全て教えてください？
Answer >  与えられた情報によると、ビールの主な原材料はホップと麦芽です。ホップはビール醸造に使用するのは花の部分の毬花部分のみで、芳香と苦味、雑菌の繁殖を抑え、ビールの腐敗を防ぐ役割があります。品質的、価格面が輸入ホップの方が優れているため輸入が主流です。麦芽は二条大麦を麦芽化させたものを使用し、ビール醸造においてビールの色や味わい、泡の形成等に大きな影響力をもつことがあります。
No:  2
Question >  ビールの製造工程の3番目の工程は？
Answer >  与えられた情報によると、ビールの製造工程の3番目の工程は「ろ過」です。
No:  3
Question >  Craft Beer Marketでは何種類のビールを提供していますか？
Answer >  与えられた情報によると、Craft Beer Marketでは以下のビアスタイルを提供しています。

1. KANPAI! STANDARD BEER（グラス：中生タンブラー パイント：UK568ml）
2. LAGER BEER / PILSNER （グラス：250ml パイント：US473ml）
3. ALE BEER （グラス：250ml パイント：US473ml）
4. WHITE BEER / WEIZEN （グラス：250ml パイント：US473ml）
5. FRUIT/HERB BEER （グラス：250ml パイント：US473ml）
6. BLACK BEER （グラス：250ml パイント：US473ml）
7. IPA – INDIA PALE ALE （グラス：250ml パイント：US473ml）
8. REAL ALE （グラス：250ml パイント：US473ml）
9. SPECIAL BEER （Wine Glass：100ml/¥480 , 200ml/¥780 ）
No:  4
Question >  Craft Beer Marketが提供するビールの中で、アルコール度数の最も高いと考えられるビールは？
Answer >  与えられた情報によると、Craft Beer Marketが提供するビー

### gpt-4o

In [47]:
%%time

LlmModel = 'gpt-4o'    # この部分のみ置き換える

chat_session_id = client.create_chat_session_on_default_collection()
print(chat_session_id)

res = []

#for i, q in enumerate(df['Question'][:3]):
for i, q in enumerate(df['Question']):
    print('No: ', i+1)
    print('Question > ', q)

    with client.connect(chat_session_id) as session:
        reply = session.query(
            message = q,
            system_prompt=system_prompt,
            pre_prompt_query=pre_prompt_query,
            prompt_query=prompt_query,
            llm=LlmModel,
            rag_config={"rag_type": "rag"},   # (Normal) RAG 
        )
        print('Answer > ', reply.content)
        res.append(reply.content)

print(len(res))

mod_name = LlmModel.replace('/', '--')
pd.DataFrame({LlmModel:res}).to_csv('../results/{}/res__{}.csv'.format(save_dir, mod_name), index=False)

59510ca5-7fef-4ead-83ca-9350f4582979
No:  1
Question >  ビールの主な原材料を全て教えてください？
Answer >  与えられた情報によると、ビールの主な原材料は以下の通りです：

1. ホップ
2. 麦芽
3. 水
4. 酵母
5. 副原料（麦、米、糖類、コーンスターチ、こうりやん、ばれいしょ、着色料）
No:  2
Question >  ビールの製造工程の3番目の工程は？
Answer >  与えられた情報によると、ビールの製造工程の3番目の工程は「ろ過」です。
No:  3
Question >  Craft Beer Marketでは何種類のビールを提供していますか？
Answer >  与えられた情報によると、Craft Beer Marketでは以下の種類のビールを提供しています：

1. KANPAI! STANDARD BEER
2. LAGER BEER / PILSNER
3. ALE BEER
4. WHITE BEER / WEIZEN
5. FRUIT/HERB BEER
6. BLACK BEER
7. IPA – INDIA PALE ALE
8. REAL ALE
9. SPECIAL BEER

合計で9種類のビールを提供しています。
No:  4
Question >  Craft Beer Marketが提供するビールの中で、アルコール度数の最も高いと考えられるビールは？
Answer >  与えられた情報によると、Craft Beer Marketが提供するビールの中でアルコール度数の最も高いと考えられるビールは「SPECIAL BEER」です。
No:  5
Question >  代表的なエールスタイルのビールを３つ教えてください？
Answer >  与えられた情報によると、代表的なエールスタイルのビールは以下の3つです：
1. アメリカン・ペールエール
2. イングリッシュ・ペールエール
3. アンバーエール
No:  6
Question >  テイスティングで、口当たりと外観はどちらを先に確認しますか？
Answer >  与えられた情報によると、テイスティングでは外観を先に確認します。
No:  7
Question >  テイスティングの際、どのような条

## Craft-Beer-Market_structured_lChunk

In [7]:
#!cat .env

In [4]:
load_dotenv(dotenv_path='.env')
heogpteKey_CBM_structured_lChunk = os.getenv('CBM_structured_lChunk')
if heogpteKey_CBM_structured_lChunk: print('key is set.')

client = H2OGPTE(
    address='https://h2ogpte.dev.h2o.ai',
    api_key=heogpteKey_CBM_structured_lChunk,
)
client

key is set.


In [5]:
save_dir = 'Craft-Beer-Market_structured_lChunk'

### gpt-4o-mini

In [6]:
%%time

LlmModel = 'gpt-4o-mini'    # この部分のみ置き換える

chat_session_id = client.create_chat_session_on_default_collection()
print(chat_session_id)

res = []

#for i, q in enumerate(df['Question'][:3]):
for i, q in enumerate(df['Question']):
    print('No: ', i+1)
    print('Question > ', q)

    with client.connect(chat_session_id) as session:
        reply = session.query(
            message = q,
            system_prompt=system_prompt,
            pre_prompt_query=pre_prompt_query,
            prompt_query=prompt_query,
            llm=LlmModel,
            rag_config={"rag_type": "rag"},   # (Normal) RAG 
        )
        print('Answer > ', reply.content)
        res.append(reply.content)

print(len(res))

mod_name = LlmModel.replace('/', '--')
pd.DataFrame({LlmModel:res}).to_csv('../results/{}/res__{}.csv'.format(save_dir, mod_name), index=False)

0b734ed9-583d-460e-956a-43a12385d0c9
No:  1
Question >  ビールの主な原材料を全て教えてください？
Answer >  与えられた情報によると、ビールの主な原材料は以下の通りです：
1. ホップ
2. 麦芽
3. 水
4. 酵母
No:  2
Question >  ビールの製造工程の3番目の工程は？
Answer >  与えられた情報によると、ビールの製造工程の3番目は「煮沸」です。この工程では、麦汁を煮沸し、ホップを加えて苦味をつけます。
No:  3
Question >  Craft Beer Marketでは何種類のビールを提供していますか？
Answer >  与えられた情報によると、Craft Beer Marketでは以下のビールスタイルを提供しています：  
1. KANPAI! STANDARD BEER  
2. LAGER BEER / PILSNER  
3. ALE BEER  
4. WHITE BEER / WEIZEN  
5. FRUIT/HERB BEER  
6. BLACK BEER  
7. IPA – INDIA PALE ALE  
8. REAL ALE  
9. SPECIAL BEER  
合計で9種類のビールスタイルがあります。
No:  4
Question >  Craft Beer Marketが提供するビールの中で、アルコール度数の最も高いと考えられるビールは？
Answer >  与えられた情報によると、Craft Beer Marketが提供するビールの中でアルコール度数が最も高いと考えられるのは「SPECIAL BEER」で、これは長期熟成でハイアルコールのビールに該当します。
No:  5
Question >  代表的なエールスタイルのビールを３つ教えてください？
Answer >  与えられた情報によると、代表的なエールスタイルのビールには「アメリカン・ペールエール」「イングリッシュ・ペールエール」「アンバーエール」があります。
No:  6
Question >  テイスティングで、口当たりと外観はどちらを先に確認しますか？
Answer >  与えられた情報によると、テイスティングの5段階では、最初に外観を確認し、その後に口当たりを評

### Qwen/Qwen2.5-72B-Instruct

In [7]:
%%time

LlmModel = 'Qwen/Qwen2.5-72B-Instruct'    # この部分のみ置き換える

chat_session_id = client.create_chat_session_on_default_collection()
print(chat_session_id)

res = []

#for i, q in enumerate(df['Question'][:3]):
for i, q in enumerate(df['Question']):
    print('No: ', i+1)
    print('Question > ', q)

    with client.connect(chat_session_id) as session:
        reply = session.query(
            message = q,
            system_prompt=system_prompt,
            pre_prompt_query=pre_prompt_query,
            prompt_query=prompt_query,
            llm=LlmModel,
            rag_config={"rag_type": "rag"},   # (Normal) RAG 
        )
        print('Answer > ', reply.content)
        res.append(reply.content)

print(len(res))

mod_name = LlmModel.replace('/', '--')
pd.DataFrame({LlmModel:res}).to_csv('../results/{}/res__{}.csv'.format(save_dir, mod_name), index=False)

25d990fb-1b20-4494-963a-6cc24ca726e4
No:  1
Question >  ビールの主な原材料を全て教えてください？
Answer >  まず、ユーザーが「ビールの主な原材料を全て教えてください？」と尋ねています。この質問に答えるためには、ドキュメントからビールの原材料に関する情報を抽出する必要があります。

ドキュメントを確認すると、名前が「1_ビールについて.pdf」の文書にビールの主な原材料が説明されています。この文書には、ホップ、麦芽、水、酵母が挙げられています。それぞれの原材料について、詳細な説明がされています。

ホップについては、つる性の植物で和名はセイヨウカラハナソウと説明されています。ビール醸造に使用するのは花の部分の毬花部分のみで、芳香と苦味、雑菌の繁殖を抑え、腐敗を防ぐ役割があるとされています。

麦芽については、二条大麦を麦芽化したもので、ビールの色や味わい、泡の形成に影響を及ぼすとされています。高温で焙煎した麦芽を使用することで香りにも影響を与えるとされています。

水については、原料で最も多く占め、工場の立地を決める重要な条件とされています。適質な水は無色、透明、無味、無臭で、生物的に汚染されていない必要があります。淡色ビールには軟水が適し、濃色ビールには硬水が適しているとされています。

酵母については、糖を分解してアルコールと炭酸ガスを生成する微生物です。ラガー酵母とエール酵母の2種類があり、それぞれ発酵温度やビールの味わいに違いがあります。

以上を踏まえて、ビールの主な原材料はホップ、麦芽、水、酵母の4つです。
</think>

与えられた情報によると、ビールの主な原材料はホップ、麦芽、水、酵母です。
No:  2
Question >  ビールの製造工程の3番目の工程は？
Answer >  まず、ユーザーが「ビールの製造工程の3番目の工程は？」と尋ねています。この質問に答えるためには、ドキュメント中の「1_ビールについて.pdf」を確認する必要があります。

ドキュメントを見ると、製造工程が順番に説明されています。最初の工程は糖化、次はろ過、そして3番目が煮沸です。煮沸工程では麦汁にホップが加えられ、苦味が作られます。この工程が3番目であることを明確にしています。

したがって、ユーザーの

### meta-llama/Meta-Llama-3.1-8B-Instruct

In [8]:
%%time

LlmModel = 'meta-llama/Meta-Llama-3.1-8B-Instruct'    # この部分のみ置き換える

chat_session_id = client.create_chat_session_on_default_collection()
print(chat_session_id)

res = []

#for i, q in enumerate(df['Question'][:3]):
for i, q in enumerate(df['Question']):
    print('No: ', i+1)
    print('Question > ', q)

    with client.connect(chat_session_id) as session:
        reply = session.query(
            message = q,
            system_prompt=system_prompt,
            pre_prompt_query=pre_prompt_query,
            prompt_query=prompt_query,
            llm=LlmModel,
            rag_config={"rag_type": "rag"},   # (Normal) RAG 
        )
        print('Answer > ', reply.content)
        res.append(reply.content)

print(len(res))

mod_name = LlmModel.replace('/', '--')
pd.DataFrame({LlmModel:res}).to_csv('../results/{}/res__{}.csv'.format(save_dir, mod_name), index=False)

1f6c7835-54d6-4dd4-8ac2-c0521c1056ac
No:  1
Question >  ビールの主な原材料を全て教えてください？
Answer >  与えられた情報によると、ビールの主な原材料はホップ、麦芽、水、酵母です。ホップはビール醸造に使用するのは花の部分の毬花部分のみで、芳香と苦味、雑菌の繁殖を抑え、ビールの腐敗を防ぐ役割があります。麦芽は二条大麦を麦芽化させたものを使用し、ビール醸造においてビールの色や味わい、泡の形成等に大きな影響力をもつ。水は原料で占める割合が一番大きく、適質の水が得られることが工場の立地を決める際の重要な条件となっています。酵母は糖を分解してアルコールと炭酸ガスを生成する神秘の微生物で、ラガー酵母とエール酵母の2つに分類されます。
No:  2
Question >  ビールの製造工程の3番目の工程は？
Answer >  与えられた情報によると、ビールの製造工程の3番目の工程は「煮沸」です。
No:  3
Question >  Craft Beer Marketでは何種類のビールを提供していますか？
Answer >  与えられた情報によると、Craft Beer Marketでは、KANPAI! STANDARD BEER、LAGER BEER / PILSNER、ALE BEER、WHITE BEER / WEIZEN、FRUIT/HERB BEER、BLACK BEER、IPA – INDIA PALE ALE、REAL ALE、SPECIAL BEERの9種類のビールを提供しています。
No:  4
Question >  Craft Beer Marketが提供するビールの中で、アルコール度数の最も高いと考えられるビールは？
Answer >  与えられた情報によると、Craft Beer Marketが提供するビールの中で、アルコール度数の最も高いと考えられるビールは「SPECIAL BEER（Wine Glass：100ml/¥480 , 200ml/¥780 ）」です。
No:  5
Question >  代表的なエールスタイルのビールを３つ教えてください？
Answer >  与えられた情報によると、代表的なエールスタイルのビールは以下の３つです。

1. アメリカン・ペールエール
2.

### gpt-4o

In [9]:
%%time

LlmModel = 'gpt-4o'    # この部分のみ置き換える

chat_session_id = client.create_chat_session_on_default_collection()
print(chat_session_id)

res = []

#for i, q in enumerate(df['Question'][:3]):
for i, q in enumerate(df['Question']):
    print('No: ', i+1)
    print('Question > ', q)

    with client.connect(chat_session_id) as session:
        reply = session.query(
            message = q,
            system_prompt=system_prompt,
            pre_prompt_query=pre_prompt_query,
            prompt_query=prompt_query,
            llm=LlmModel,
            rag_config={"rag_type": "rag"},   # (Normal) RAG 
        )
        print('Answer > ', reply.content)
        res.append(reply.content)

print(len(res))

mod_name = LlmModel.replace('/', '--')
pd.DataFrame({LlmModel:res}).to_csv('../results/{}/res__{}.csv'.format(save_dir, mod_name), index=False)

f0377cdd-0058-4680-9e14-720d7208c10d
No:  1
Question >  ビールの主な原材料を全て教えてください？
Answer >  与えられた情報によると、ビールの主な原材料は以下の通りです：

1. ホップ
2. 麦芽（二条大麦）
3. 水
4. 酵母
No:  2
Question >  ビールの製造工程の3番目の工程は？
Answer >  与えられた情報によると、ビールの製造工程の3番目の工程は「煮沸」です。
No:  3
Question >  Craft Beer Marketでは何種類のビールを提供していますか？
Answer >  与えられた情報によると、Craft Beer Marketでは以下のビールスタイルを提供しています：

1. KANPAI! STANDARD BEER
2. LAGER BEER / PILSNER
3. ALE BEER
4. WHITE BEER / WEIZEN
5. FRUIT/HERB BEER
6. BLACK BEER
7. IPA – INDIA PALE ALE
8. REAL ALE
9. SPECIAL BEER

以上、9種類のビールスタイルを提供しています。
No:  4
Question >  Craft Beer Marketが提供するビールの中で、アルコール度数の最も高いと考えられるビールは？
Answer >  与えられた情報によると、Craft Beer Marketが提供するビールの中でアルコール度数の最も高いと考えられるビールは「SPECIAL BEER」です。
No:  5
Question >  代表的なエールスタイルのビールを３つ教えてください？
Answer >  与えられた情報によると、代表的なエールスタイルのビールは以下の3つです：
1. アメリカン・ペールエール
2. イングリッシュ・ペールエール
3. アンバーエール
No:  6
Question >  テイスティングで、口当たりと外観はどちらを先に確認しますか？
Answer >  与えられた情報によると、テイスティングでは外観を先に確認します。
No:  7
Question >  テイスティングの際、どのような条件の時、香りが強くなるか？
Answer >  与えられ

In [1]:
!ls -a

.                               .ipynb_checkpoints
..                              Ans_in_prompt.ipynb
.env                            Craft-Beer-Market_run_RAG.ipynb


In [3]:
!cat .env

CBM_default=sk-jej37QUsu8bVJ66NcAqrssiyw8pYhfEy1EsC5k6CHzo6BG1W
CBM_structured=sk-9YIjkqdjBEKktzaCVfYzaZSCnRu1D9eoAP4H0tSoJleXA9hq
CBM_structured_lChunk=sk-aGA8dRY7s93bfAeQS5PepxGVo8SkwVr7zgOHxyYAFIO1o8kx
CBM_ref_prompt_NoRAG=sk-OFj9aHMrjMfbh54SX5ONhaFBYPywZmETT5vwbCwpIdhlOTdz
H2OGPTE_GLOBAL_KEY=sk-mcgXgu2SZ6X6cDwfHrxsrPift3nhvZhVbp2vs5NEh7pmc6ER


In [5]:
load_dotenv(dotenv_path='.env')
heogpteKey = os.getenv('CBM_ref_prompt_NoRAG')
if heogpteKey: print('key is set.')

client = H2OGPTE(
    address='https://h2ogpte.dev.h2o.ai',
    api_key=heogpteKey,
)
client

NameError: name 'load_dotenv' is not defined